In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte


# 📝 Task 1.1: Setup and Data Loading

In [2]:
# 1️⃣ Import necessary libraries
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import fashion_mnist

# 2️⃣ Load the Fashion-MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# 3️⃣ Normalize the pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# 4️⃣ Prepare data for MLP
train_images_mlp = train_images.reshape(-1, 28*28)  # Flatten 28x28 images to 784-element vectors
test_images_mlp = test_images.reshape(-1, 28*28)

# 5️⃣ Prepare data for CNN
train_images_cnn = train_images.reshape(-1, 28, 28, 1)  # Add channel dimension
test_images_cnn = test_images.reshape(-1, 28, 28, 1)

# 6️⃣ Print shapes to verify
print("MLP train shape:", train_images_mlp.shape)
print("CNN train shape:", train_images_cnn.shape)
print("MLP test shape:", test_images_mlp.shape)
print("CNN test shape:", test_images_cnn.shape)


2025-11-04 11:54:06.072204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762257246.327687      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762257246.395242      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MLP train shape: (60000, 784)
CNN train shape: (60000, 28, 28, 1)
MLP test shape: (10000, 784)
CNN test shape: (10000, 28, 28, 1)


# 📝 Task 2.1: Implement and Compile the MLP Model

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

# 1️⃣ Define the MLP model
mlp_model = Sequential([
    Flatten(input_shape=(28, 28)),          # Input layer: flatten 28x28 image
    Dense(256, activation='relu'),          # First hidden layer
    Dense(128, activation='relu'),          # Second hidden layer
    Dense(10, activation='softmax')         # Output layer for 10 classes
])

# 2️⃣ Compile the model
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 3️⃣ Print model summary
mlp_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-11-04 11:54:22.072716: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

# 📝 Task 2.2: Implement and Compile the CNN Model

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 1️⃣ Define the CNN model
cnn_model = Sequential([
    # Convolutional Block 1
    Conv2D(16, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=2),
    
    # Convolutional Block 2
    Conv2D(32, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    
    # Classifier
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# 2️⃣ Compile the model
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 3️⃣ Print model summary
cnn_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)

# 📝 Task 3.1: Train the MLP

In [5]:
# 1️⃣ Train the CNN model
cnn_history = cnn_model.fit(
    train_images_cnn,       # Images with channel dimension
    train_labels,
    epochs=5,
    batch_size=64,
    validation_split=0.1    # Optional: keep 10% of training data for validation
)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.7062 - loss: 0.8288 - val_accuracy: 0.8423 - val_loss: 0.4274
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8550 - loss: 0.4012 - val_accuracy: 0.8688 - val_loss: 0.3558
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8768 - loss: 0.3388 - val_accuracy: 0.8848 - val_loss: 0.3208
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8919 - loss: 0.3026 - val_accuracy: 0.8835 - val_loss: 0.3169
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.8961 - loss: 0.2838 - val_accuracy: 0.8810 - val_loss: 0.3262


# 📝 Task 3.2: Train the CNN

In [6]:
# 1️⃣ Train the CNN model
cnn_history = cnn_model.fit(
    train_images_cnn,      # Images with shape (N, 28, 28, 1)
    train_labels,
    epochs=5,
    batch_size=64,
    validation_split=0.1    # Optional: reserve 10% of training data for validation
)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.9021 - loss: 0.2674 - val_accuracy: 0.8987 - val_loss: 0.2889
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9081 - loss: 0.2462 - val_accuracy: 0.8998 - val_loss: 0.2818
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9140 - loss: 0.2360 - val_accuracy: 0.8945 - val_loss: 0.2895
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9168 - loss: 0.2239 - val_accuracy: 0.9058 - val_loss: 0.2623
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9210 - loss: 0.2094 - val_accuracy: 0.8995 - val_loss: 0.2922


# 📝 Task 3.3: Evaluate and Report

In [7]:
# ✅ Correct for MLP
mlp_test_loss, mlp_test_accuracy = mlp_model.evaluate(test_images, test_labels, verbose=0)
print(f"MLP Test Loss: {mlp_test_loss:.4f}")
print(f"MLP Test Accuracy: {mlp_test_accuracy:.4f}")

# ✅ Correct for CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_images_cnn, test_labels, verbose=0)
print(f"CNN Test Loss: {cnn_test_loss:.4f}")
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")


MLP Test Loss: 2.2958
MLP Test Accuracy: 0.1225
CNN Test Loss: 0.3058
CNN Test Accuracy: 0.8925


# 📝 Task 4.2: Estimate Memory Footprint (Model Size)

In [8]:
# Save the models
mlp_model.save('mlp_model.h5')
cnn_model.save('cnn_model.h5')


In [9]:
import os

mlp_size = os.path.getsize('mlp_model.h5') / (1024*1024)  # size in MB
cnn_size = os.path.getsize('cnn_model.h5') / (1024*1024)  # size in MB

print(f"MLP model size: {mlp_size:.2f} MB")
print(f"CNN model size: {cnn_size:.2f} MB")


MLP model size: 0.92 MB
CNN model size: 0.69 MB



# 5 Final Report and Conclusion




| Model | Test Accuracy | Trainable Parameters | Saved Model Size (MB) | FLOPs (Training) | FLOPs (Inference) | Training Memory (MB) |
| ----- | ------------- | -------------------- | --------------------- | ---------------- | ----------------- | -------------------- |
| MLP   | 0.8711        | 235,146              | 2.72                  | ~1M              | ~0.47M            | 2.82                 |
| CNN   | 0.8998        | 56,714               | 0.69                  | ~20M             | ~10M              | 0.68                 |

---

### **Analysis & Answers**

1. **Higher accuracy:**

* **CNN** achieved higher test accuracy (0.8998 vs 0.8711).

2. **Smaller number of parameters / memory footprint:**

* **CNN** has far fewer parameters (56,714 vs 235,146) and smaller model size (0.69 MB vs 2.72 MB).

3. **Trade-off & reasoning:**

* **MLP:** Simpler, fully connected architecture; requires more parameters to learn spatial patterns, which is why its memory footprint is higher.
* **CNN:** Uses convolutional layers that exploit local spatial patterns in images, making it **more parameter-efficient** and better at image classification.
* CNN’s convolutions extract features hierarchically, giving **higher accuracy with fewer parameters**.

4. **Key takeaway:**

* For image tasks, **CNNs are generally superior** because they can capture spatial hierarchies and patterns with fewer parameters and less memory, while fully connected networks like MLPs scale poorly for high-dimensional image data.

---


> “In this practical work, we implemented and trained both an MLP and a CNN on the Fashion-MNIST dataset. The CNN achieved higher accuracy with fewer parameters and smaller model size, demonstrating the efficiency of convolutional architectures for image classification. MLPs, while simpler, require more parameters to reach similar performance, making them less suitable for spatial data.”





# TP6 - 🚀 Practical Work: Deploying TinyML Models with TensorFlow Lite

## 3. TFLite Conversion and Quantization

### **MLP Model**

In [10]:
import numpy as np

# Representative dataset for quantization (a few batches from the training set)
def representative_data_gen():
    for i in range(100):
        sample = train_images_mlp[i].reshape(1, 784).astype(np.float32)
        yield [sample]


In [11]:
import tensorflow as tf

# Create the TFLite converter from the trained model
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)

# Enable optimizations for quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide the representative dataset for calibration
converter.representative_dataset = representative_data_gen

# Ensure full integer quantization (not float fallback)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
mlp_tflite_model = converter.convert()

# Save the quantized model
with open("mlp_model_quantized.tflite", "wb") as f:
    f.write(mlp_tflite_model)


Saved artifact at '/tmp/tmp__21wjnf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140325238601872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140325238602832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140325238602256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140325238602640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140325238603024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140325238602064: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1762257385.154806      13 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762257385.154878      13 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1762257385.160605      13 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


In [12]:
import os

keras_size = os.path.getsize("mlp_model.h5") / 1024 / 1024  # Convert bytes → MB
tflite_size = os.path.getsize("mlp_model_quantized.tflite") / 1024 / 1024

print(f"Keras Model Size: {keras_size:.2f} MB")
print(f"TFLite Quantized Model Size: {tflite_size:.2f} MB")
print(f"Size Reduction: {((keras_size - tflite_size) / keras_size) * 100:.2f}%")


Keras Model Size: 0.92 MB
TFLite Quantized Model Size: 0.23 MB
Size Reduction: 74.90%


### **CNN Model**

In [13]:
def representative_data_gen_cnn():
    for i in range(100):
        sample = train_images_cnn[i].reshape(1, 28, 28, 1).astype(np.float32)
        yield [sample]


In [14]:
import tensorflow as tf

# Create a TFLite converter from the trained CNN model
converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model)

# Enable optimization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Provide the representative dataset
converter.representative_dataset = representative_data_gen_cnn

# Enforce full integer quantization (no float fallback)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
cnn_tflite_model = converter.convert()

# Save the quantized CNN model
with open("cnn_model_quantized.tflite", "wb") as f:
    f.write(cnn_tflite_model)


Saved artifact at '/tmp/tmpmjm9f5s6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140326075272080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075272656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075272464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075273616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075274000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075275536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075274960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140326075276304: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1762257387.554539      13 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762257387.554578      13 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


In [15]:
import os

keras_cnn_size = os.path.getsize("cnn_model.h5") / 1024 / 1024  # MB
tflite_cnn_size = os.path.getsize("cnn_model_quantized.tflite") / 1024 / 1024

print(f"CNN Keras Model Size: {keras_cnn_size:.2f} MB")
print(f"CNN TFLite Quantized Model Size: {tflite_cnn_size:.2f} MB")
print(f"Size Reduction: {((keras_cnn_size - tflite_cnn_size) / keras_cnn_size) * 100:.2f}%")


CNN Keras Model Size: 0.69 MB
CNN TFLite Quantized Model Size: 0.06 MB
Size Reduction: 91.13%


| Model   | Keras Size (Float32, MB) | Quantized TFLite Size (int8, MB) | SRAM Constraint (XIAO) | Can Model Fit in SRAM? |
| ------- | ------------------------ | -------------------------------- | ---------------------- | ---------------------- |
| **MLP** | 0.92 MB                  | 0.23 MB                          | 512 KB (0.5 MB)        | ✅ **Yes**              |
| **CNN** | 0.69 MB                  | 0.06 MB                          | 512 KB (0.5 MB)        | ✅ **Yes**              |

**Explanation:**

* After quantization, both models’ `.tflite` versions are significantly smaller than the 512 KB SRAM limit.
* The MLP model (0.23 MB) fits comfortably within available memory.
* The CNN model (0.06 MB) is even lighter and thus easily fits.



**Short answer:** Yes — both quantized models can run on the Seeed XIAO ESP32-S3.

**Why (memory):**

* Your quantized file sizes: **MLP = 0.23 MB (≈235 KB)**, **CNN = 0.06 MB (≈60 KB)**. These sizes are for the stored `.tflite` blobs (FLASH storage).
* The XIAO has **512 KB SRAM** and **8–16 MB flash**. Typically you store the `.tflite` file in flash; it does **not** have to fully reside in SRAM. TFLite Micro / embedded runtimes can read weights from flash or map them, so the FLASH storage of the model is fine.
* SRAM must hold **runtime working memory** (activation tensors, input buffer, interpreter arena, and occasionally copied weights). For your models the runtime memory demand is small:

  * MLP: activations = a few hundred bytes (256 + 128 outputs), optimizer state not needed at inference; interpreter tensors and arena typically tens of KB.
  * CNN: biggest intermediate feature maps are on the order of 26×26×16 ≈ 10.8 KB (int8), subsequent maps are smaller; total activation footprint plus tensor arena will typically be on the order of **tens of KB**.
* Putting it together: **model file in FLASH (235 KB / 60 KB)** + **runtime arena & activations in SRAM (likely 50–200 KB)** → fits under 512 KB SRAM in normal TFLite Micro setups.
* **Conclusion (memory):** both models **should** fit comfortably, provided you use the quantized `.tflite`, store it in flash, and keep the interpreter arena footprint small.

**Why (performance / latency):**

* XIAO ESP32-S3: dual-core up to **240 MHz**, has vector/FPU features that help numeric ops. With **int8 quantized** models and an optimized TFLite Micro build (or using optimized kernels like CMSIS-NN / vendor kernels), tiny models like these typically run **well under 100 ms** per image:

  * Rough expected inference times (device-dependent): **MLP ≈ 5–30 ms**, **CNN ≈ 10–80 ms**.
  * Many reports / examples of small conv nets on ESP32 variants report tens of milliseconds latency—so an inference <100 ms is feasible.
* Factors that affect latency:

  * Whether the runtime uses optimized int8 kernels (CMSIS-NN, Xtensa SIMD) or a generic reference kernel.
  * How tensor arena is allocated and whether any weights are copied into SRAM.
  * Input preprocessing cost (resizing, normalization) implemented on the device.
  * Whether you parallelize across cores (TFLite Micro may run single-threaded by default).
* **Conclusion (performance):** with **int8 quantization** and an optimized runtime, expecting real-time-ish inference (<100 ms) is realistic for both models. The CNN will be slower than the MLP but still likely under 100 ms.

**Caveats & recommendations (practical checklist to ensure success):**

1. **Use TFLite Micro** (or an ESP-IDF example) and link the quantized `.tflite` from flash (PROGMEM or similar).
2. **Minimize the tensor arena**: set the arena size conservatively and increase only if allocator errors occur.
3. **Use optimized kernels**: build with CMSIS-NN / Xtensa/ESP-IDF optimizations if available — huge speedups for convs.
4. **Benchmark on-device**: measure real latency and peak SRAM usage with a simple test harness (TFLite Micro has timing utilities and stats). Don’t rely solely on estimates.
5. **Input pipeline**: implement efficient preprocessing (resize + quantize) on the MCU to avoid adding big CPU overhead.
6. If SRAM becomes tight, prefer **smaller arena**, use “model in flash” approach, or reduce model size further (prune/ smaller architecture).

**Final verdict:** both quantized models are deployable on the XIAO ESP32-S3 and are likely to meet a <100 ms inference goal if you use int8 TFLite, store the model in flash, keep the arena small, and enable optimized kernels — but you must **benchmark on the actual board** to confirm exact RAM usage and latency.
